교체하는 작업은 dict를 사용해서 key:value 형태로 교체하면 그만일 듯

In [1]:
import numpy as np
import pandas as pd

import os
import datetime

In [2]:
money_data = pd.read_csv("./data/2021_money.csv", encoding="EUC-KR", header=0)
train_data = pd.read_csv("./data/2021_subway_transport_data.csv", encoding="EUC-KR", header=0)
money_data.head()

,기준_년_코드,기준_분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,서비스_업종_코드,서비스_업종_코드_명,분기당_매출_금액,분기당_매출_건수,...,시간대_건수~24_매출_건수,남성_매출_건수,여성_매출_건수,연령대_10_매출_건수,연령대_20_매출_건수,연령대_30_매출_건수,연령대_40_매출_건수,연령대_50_매출_건수,연령대_60_이상_매출_건수,점포수
0,2021,1,A,골목상권,2110001,이북5도청사,CS100001,한식음식점,626851391,25502,...,157,15667,7253,108,1098,2013,4344,7792,7568,28
1,2021,1,A,골목상권,2110001,이북5도청사,CS100008,분식전문점,122957138,12039,...,0,6679,4293,52,926,1222,2698,3404,2670,9
2,2021,1,A,골목상권,2110001,이북5도청사,CS100009,호프-간이주점,5093349,177,...,9,124,36,0,0,9,18,62,71,3
3,2021,1,A,골목상권,2110001,이북5도청사,CS100010,커피-음료,13507733,1359,...,8,356,881,0,243,260,213,288,235,5
4,2021,1,A,골목상권,2110001,이북5도청사,CS200001,일반교습학원,37551380,128,...,0,56,72,0,0,8,96,8,16,3


In [3]:
train_data.head()

,연번,날짜,호선,역번호,역명,구분,06시 이전,06시-07시,07시-08시,08시-09시,...,15시-16시,16시-17시,17시-18시,18시-19시,19시-20시,20시-21시,21시-22시,22시-23시,23시 이후,합 계
0,1,2021-01-01,1,150,서울역,승차,86,111,157,306,...,837,796,839,658,579,479,510,445,101,9339
1,2,2021-01-01,1,150,서울역,하차,85,355,438,592,...,781,710,626,437,425,354,307,134,77,9951
2,3,2021-01-01,1,151,시청,승차,43,45,75,111,...,227,233,347,294,186,216,204,110,29,2947
3,4,2021-01-01,1,151,시청,하차,40,145,176,312,...,158,140,152,121,57,54,64,23,17,2700
4,5,2021-01-01,1,152,종각,승차,49,59,89,126,...,513,548,504,572,492,593,513,130,31,5686


In [4]:
print(len(money_data[money_data['상권_코드_명'].str.contains("역")]))
print(len(money_data[money_data['상권_코드_명'].str.contains(r".역")]))

54448
54232


In [5]:
# money_data[money_data['상권_코드_명'].str.contains(r".역")]['상권_코드_명'].unique()  # 정규표현식 사용해서 ~~역으로 끝나는 것만 추출

In [6]:
station_money = pd.DataFrame(money_data[money_data['상권_코드_명'].str.contains("역")]['상권_코드_명'].unique())
station_money.columns = ["상권코드명",]
# print(station_money.iloc[1], type(station_money.iloc[0]))
# print(station_money.iloc[1].astype('str').str.split('역'))

# tmp = station_money.iloc[1].astype('str').str.split('역')
# print(len(tmp[0]))
# print(tmp.str.get(0))

# print(len(station_money))

# for i in range(len(station_money)):
#     splited = station_money.iloc[i].astype('str').str.split("역")
#     for j in range(len(splited[0])):
#         col_name = str(j+1) + "번 분리 데이터" 
#         if col_name in station_money.columns:
#             continue
#         else:
#             station_money[col_name] = None
#     for j in range(len(splited[0])):
#         col_name = str(j+1) + "번 분리 데이터" 
#         station_money[col_name].iloc[i] = splited.str.get(j)

col_arr = []

splt = station_money["상권코드명"].str.split(r"역$|역 |역\(|역,|역_|역골|역지|역전|역1|\(| |역상|역둘|역동|역\)|역교", expand=True)
for i in range(len(splt.columns)):
    col_name = str(i+1) + "번 분리 데이터" 
    col_arr.append(col_name)
splt.columns = col_arr

origin_station_money = station_money.copy()
station_money = pd.concat([station_money, splt], axis=1)

# station_money
station_money.to_csv("./station_data_moneydata.csv", encoding="utf-8-sig")

In [7]:
station_train = pd.DataFrame(train_data['역명'].unique())
station_train.columns = ["역명",]
# station_train

col_arr_train = []

splt_train = station_train["역명"].str.split(r"\(|역$", expand=True)
for i in range(len(splt_train.columns)):
    col_name = str(i+1) + "번 분리 데이터"
    col_arr_train.append(col_name)
splt_train.columns = col_arr_train

origin_station_train = station_train.copy()
station_train = pd.concat([station_train, splt_train], axis=1)

station_train.to_csv("./station_data_traindata.csv", encoding="utf-8-sig")

In [8]:
# station_money['1번 분리 데이터'].unique()
# '서울' in station_money['1번 분리 데이터'].unique()

In [9]:
# station_train['1번 분리 데이터'].unique()
# '서울' in station_train['1번 분리 데이터'].unique()

In [10]:
# stm_arr = station_money['1번 분리 데이터'].unique()
# stt_arr = station_train['1번 분리 데이터'].unique()

# stm_arr_origin = stm_arr.copy()
# stt_arr_origin = stt_arr.copy()

# # print(type(stm_arr))
# # print(len(stm_arr))

# matching_idx_arr = []

# for i in range(len(stt_arr)):
#     # print(len(stt_arr))
#     for j in range(len(stm_arr)):
#         # print(len(stm_arr))
#         if stt_arr[i] == stm_arr[j]:
#             matching_idx_arr.append(j)
#             stm_arr = np.delete(stm_arr, j)
#             break
# print(matching_idx_arr)
# stm_arr

# # 이런 느낌으로.. 분리한 데이터를 

In [11]:
# 예상 구동시간 20초
stt_arr = station_train['1번 분리 데이터'].unique()

# pop 사용, 실제 pop는 column이니까, concat으로
poped_df = pd.DataFrame()

for i in range(len(stt_arr)):
    if stt_arr[i] == '둔촌동':
        stt_arr[i] = '둔촌' or '둔촌동'
    for j in range(len(station_money.columns)):
        for k in range(len(station_money)):
            if station_money.iloc[k, j] == stt_arr[i]:
                poped = pd.DataFrame(station_money.iloc[k]).T
                poped['idx'] = j
                poped_df = pd.concat([poped_df, poped], axis=0, ignore_index=True)
                station_money.drop([station_money.index[k]], inplace=True, axis=0)
                break
            
station_money
# 341 312 122 120 120 118 118 118

,상권코드명,1번 분리 데이터,2번 분리 데이터,3번 분리 데이터,4번 분리 데이터
4,동대문역 3번,동대문,3번,None,None
6,창신역 4번,창신,4번,None,None
8,동묘앞역 3번,동묘앞,3번,None,None
14,청구역 3번,청구,3번,None,None
17,효창공원앞역 5번,효창공원앞,5번,None,None
...,...,...,...,...,...
537,선유도역 골목형상점가,선유도,골목형상점가,None,None
538,영등포역지하도상점가,영등포,하도상점가,None,None
539,남성역골목시장,남성,목시장,None,None
540,강남터미널지하도상점가2구역,강남터미널지하도상점가2구,,None,None


In [12]:
# 예상 구동시간 50초
prelen = 0
cnt = 0
loops = 0

while len(station_money) != prelen or cnt < 3:
    if loops != 0:
        prelen = len(station_money)
    # print("이전길이", prelen)

    for i in range(len(stt_arr)):
        if stt_arr[i] == '둔촌동':
            stt_arr[i] = '둔촌' or '둔촌동'
        for j in range(len(station_money.columns)):
            for k in range(len(station_money)):
                if station_money.iloc[k, j] == stt_arr[i]:
                    poped = pd.DataFrame(station_money.iloc[k]).T
                    poped['idx'] = j
                    poped_df = pd.concat([poped_df, poped], axis=0, ignore_index=True)
                    station_money.drop([station_money.index[k]], inplace=True, axis=0)
                    break
    
    if prelen == len(station_money):
        cnt += 1
    else:
        cnt = 0

    loops += 1
    # print("현재길이",len(station_money))
    # print("반복횟수", cnt)

# 남은것 보기
station_money
# 217 151 127 123 121 121 112

,상권코드명,1번 분리 데이터,2번 분리 데이터,3번 분리 데이터,4번 분리 데이터
20,남영역 1번,남영,1번,None,None
25,서빙고역 1번,서빙고,1번,None,None
38,서울숲역 1번,서울숲,1번,None,None
57,회기역 1번,회기,1번,None,None
58,외대앞역 1번,외대앞,1번,None,None
...,...,...,...,...,...
511,송파나루역,송파나루,,None,None
536,역촌중앙시장,역촌중앙시장,None,None,None
537,선유도역 골목형상점가,선유도,골목형상점가,None,None
538,영등포역지하도상점가,영등포,하도상점가,None,None


In [13]:
# poped_df

In [14]:
station_money.to_csv("./station_data_testing.csv", encoding='utf-8-sig')
# poped_df.to_csv("./poped_data.csv", encoding='utf-8-sig')

In [15]:
stt_dict = dict(zip(stt_arr, range(len(stt_arr))))

# # stt_dict
# stt_dict['남위례']

In [16]:
poped_df_origin = poped_df.copy()
poped_df['link'] = None

for i in range(len(poped_df)):
    col_idx = poped_df.iloc[i]['idx']
    linked_data = poped_df.iloc[i, col_idx]
    
    poped_df.iloc[i, -1] = stt_dict[linked_data]

poped_df

,상권코드명,1번 분리 데이터,2번 분리 데이터,3번 분리 데이터,4번 분리 데이터,idx,link
0,서울역 15번,서울,15번,None,None,1,0
1,시청역_1번,시청,1번,None,None,1,1
2,북창동(시청역_6번),북창동,시청,6번),None,2,1
3,종각역,종각,,None,None,1,2
4,종로3가역,종로3가,,None,None,1,3
...,...,...,...,...,...,...,...
425,굽은다리역,굽은다리,,None,None,1,144
426,내방역 1번,내방,1번,None,None,1,206
427,암사역,암사,,None,None,1,230
428,연신내역,연신내,,None,None,1,60


In [17]:
poped_df.to_csv("./poped_data.csv", encoding='utf-8-sig')

In [18]:
linked_df = poped_df.loc[:,['상권코드명', 'link']]

linked_df

,상권코드명,link
0,서울역 15번,0
1,시청역_1번,1
2,북창동(시청역_6번),1
3,종각역,2
4,종로3가역,3
...,...,...
425,굽은다리역,144
426,내방역 1번,206
427,암사역,230
428,연신내역,60


In [19]:
linked_df.to_csv("./sales_district_name_IDcode.csv", encoding='utf-8-sig')

In [20]:
stt_df = station_train

if 'link' in stt_df.columns:
    del stt_df['link']

stt_df['link'] = None
for i in range(len(stt_df)):
    linked_data = stt_df.iloc[i]['1번 분리 데이터']
    
    if linked_data == '둔촌동':
        linked_data = '둔촌' or '둔촌동'
        
    stt_df.iloc[i, -1] = stt_dict[linked_data]
    
stt_df

,역명,1번 분리 데이터,2번 분리 데이터,link
0,서울역,서울,,0
1,시청,시청,None,1
2,종각,종각,None,2
3,종로3가,종로3가,None,3
4,종로5가,종로5가,None,4
...,...,...,...,...
244,신내,신내,None,244
245,강일,강일,None,245
246,하남검단산,하남검단산,None,246
247,하남시청(덕풍·신장),하남시청,덕풍·신장),247


기준_년_코드	기준_분기_코드	상권_구분_코드	상권_구분_코드_명	상권_코드	상권_코드_명	서비스_업종_코드	서비스_업종_코드_명	분기당_매출_금액	분기당_매출_건수	...	시간대_건수~24_매출_건수	남성_매출_건수	여성_매출_건수	연령대_10_매출_건수	연령대_20_매출_건수	연령대_30_매출_건수	연령대_40_매출_건수	연령대_50_매출_건수	연령대_60_이상_매출_건수	점포수

In [26]:
stt_df.to_csv("./subway_station_name_IDcode.csv", encoding='utf-8-sig')

In [36]:
linked_stt_df = stt_df[['역명', 'link']]
linked_stt_df.to_csv("./subway_station_name_IDcode.csv", encoding='utf-8-sig')

In [35]:
# testing_data = money_data[money_data['상권_코드_명'].str.contains("역")].iloc[0:100]
testing_data = money_data[money_data['상권_코드_명'].str.contains("역")].iloc[0:10]

if 'link' in testing_data.columns:
    del testing_data['link']

testing_data['link'] = None

# 추가
testing_data.reset_index(drop=True, inplace=True)

# 추가 종료

for i in range(len(testing_data)):
    for j in range(len(linked_df)):
        # if testing_data.iloc[i, 5] == linked_df.iloc[j, 0]:
        if testing_data.loc[i, '상권_코드_명'] == linked_df.iloc[j, 0]:
            # testing_data.iloc[i, -1] = linked_df.iloc[j, 1]
            testing_data.loc[i, 'link'] = linked_df.iloc[j, 1]

result_df = testing_data[['상권_코드_명', '서비스_업종_코드_명', '점포수', 'link']]

result_df
# 상권_코드_명

,상권_코드_명,서비스_업종_코드_명,점포수,link
0,독립문역 1번,한식음식점,29,65
1,독립문역 1번,제과점,9,65
2,독립문역 1번,치킨전문점,9,65
3,독립문역 1번,커피-음료,9,65
4,독립문역 1번,일반교습학원,12,65
5,독립문역 1번,외국어학원,9,65
6,독립문역 1번,치과의원,3,65
7,독립문역 1번,골프연습장,1,65
8,독립문역 1번,PC방,3,65
9,독립문역 1번,자동차수리,4,65


,날짜,호선,역번호,역명,구분,
06시 이전,06시-07시,07시-08시,08시-09시,09시-10시,10시-11시,
11시-12시,12시-13시,13시-14시,14시-15시,15시-16시,16시-17시,
17시-18시,18시-19시,19시-20시,20시-21시,21시-22시,22시-23시,23시 이후,
합 계,0시11시,11시17시,17시24시,06시11시,연,월,일,요일

In [22]:
# 예상시간 6분 30초
arrive_data = pd.read_csv("./data/2021_arrive_qt_tz_01.csv", encoding='utf-8')
linked_stt_df = stt_df[['역명', 'link']]
# linked_stt_df

arrive_data_origin = arrive_data.copy()

if 'link' in arrive_data.columns:
    del arrive_data['link']

arrive_data['link'] = None

for i in range(len(arrive_data)):
    for j in range(len(linked_stt_df)):
        # if arrive_data.iloc[i, 4] == linked_stt_df.iloc[j, 0]:
        if arrive_data.loc[i, '역명'] == linked_stt_df.iloc[j, 0]:
            # arrive_data.iloc[i, -1] = linked_stt_df.iloc[j, 1]
            arrive_data.loc[i, 'link'] = linked_stt_df.iloc[j, 1]

result_stt_df = arrive_data[['역명','00시11시','11시17시','17시24시','합 계','link']]

result_stt_df

,역명,00시11시,11시17시,17시24시,합 계,link
0,서울역,3058,4533,2360,9951,0
1,시청,1085,1127,488,2700,1
2,종각,1385,2601,1001,4987,2
3,종로3가,1050,3127,889,5066,3
4,종로5가,756,2342,639,3737,4
...,...,...,...,...,...,...
24999,남한산성입구(성남법원.검찰청),2692,3190,7136,13018,239
25000,단대오거리,2351,2525,5612,10488,240
25001,신흥,932,1276,2520,4728,241
25002,수진,979,1306,2466,4751,242


In [23]:
merge_test_df = testing_data[['상권_코드_명', '서비스_업종_코드_명', '점포수', '분기당_매출_건수', 'link']]
merge_test_stt_df = arrive_data[['역명','00시11시','11시17시','17시24시','합 계','link']]

In [24]:
# final_result = pd.DataFrame()

# for i in range(len(merge_test_df)):
#     money_target = merge_test_df.iloc[i].T
#     for j in range(len(merge_test_stt_df)):
#         people_target = merge_test_stt_df.iloc[j].T
#         if money_target['link'] == people_target['link']:
#             final_result = pd.concat([final_result,money_target,people_target],axis=1, ignore_index=True)

# final_result

In [28]:
merge_test_df = testing_data[['상권_코드_명', '서비스_업종_코드_명', '점포수', '분기당_매출_건수', 'link']]
# merge_test_df.columns = ['상권_코드_명', '서비스_업종_코드_명', '점포수', '분기당_매출_건수', 'linkID']
merge_test_df = pd.DataFrame(merge_test_df[merge_test_df['서비스_업종_코드_명'].str.contains("제과")])
# merge_test_df = pd.DataFrame(merge_test_df[merge_test_df['상권_코드_명'].str.contains("독립문")])
# station_money = pd.DataFrame(money_data[money_data['상권_코드_명'].str.contains("역")]['상권_코드_명'].unique())

merge_test_stt_df = arrive_data[['역명','00시11시','11시17시','17시24시','합 계','link']]
# merge_test_stt_df.columns = ['역명','00시11시','11시17시','17시24시','합 계','linkID']
# merge_test_stt_df.index[(merge_test_stt_df['link']==65) | (merge_test_stt_df['link']==174)]
# merge_test_stt_df['link'][[67,198]]
# print(merge_test_df)
merge_test_stt_df = merge_test_stt_df.iloc[[67,198],:]

# print (df.index[(df['B'] == 19) | (df['C'] == 19)].tolist())

tmp_df = pd.DataFrame()
final_result = pd.DataFrame()

tmp_df = pd.merge(merge_test_df, merge_test_stt_df, how='outer', on='link')
final_result = pd.concat([final_result, tmp_df],axis=1)
        # final_result = pd.concat([final_result, people_target],axis=1, ignore_index=True)

final_result

,상권_코드_명,서비스_업종_코드_명,점포수,분기당_매출_건수,link,역명,00시11시,11시17시,17시24시,합 계
0,독립문역 1번,제과점,9,50998,65,독립문,506,925,634,2065
1,창신역 1번,제과점,3,18119,174,창신,132,264,313,709


In [56]:
#예상시간 1분 10초
bakery_data = money_data[money_data['서비스_업종_코드_명'].str.contains("제과")]
bakery_data.reset_index(drop=True, inplace=True)
# bakery_data

if 'link' in bakery_data.columns:
    del bakery_data['link']

bakery_data['link'] = None

for i in range(len(bakery_data)):
    for j in range(len(linked_df)):
        if bakery_data.loc[i, '상권_코드_명'] == linked_df.iloc[j, 0]:
            bakery_data.loc[i, 'link'] = linked_df.iloc[j, 1]

C:\Users\La_Elya\AppData\Local\Temp\ipykernel_6796\2739844580.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bakery_data['link'] = None


In [ ]:
bakery_data.head()
len(bakery_data)

2935

In [ ]:
# 분기별로 매출 나누기
# tsda = bakary_data.iloc[0:100]
bd_qt1 = bd_qt2 = bd_qt3 = bd_qt4 = pd.DataFrame()

for i in range(len(bakery_data)):
    target_row = pd.DataFrame(bakery_data.iloc[i]).T
    target_qt = bakery_data.loc[i, '기준_분기_코드']
    for j in range(4):
        if target_qt == j+1:
            globals()["bd_qt"+str(j+1)] = pd.concat([globals()["bd_qt"+str(j+1)], target_row], axis=0, ignore_index=True)


In [ ]:
# arrive_data = pd.read_csv("./data/2021_arrive_qt_tz_01.csv", encoding='utf-8')
bd_qt1['link'].value_counts()

118    4
63     4
150    4
34     4
92     3
      ..
174    1
3      1
67     1
2      1
133    1
Name: link, Length: 172, dtype: int64